In [1]:
import pandas as pd
import gdown
import re
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from tqdm import tqdm

In [2]:
def driver_setup():# Assigning a Headless Firefox Driver
    options = webdriver.FirefoxOptions()
    options.binary_location = 'C:/Program Files/Mozilla Firefox/firefox.exe'  # Path to Chrome executable
    options.add_argument('E:/Programs/geckodriver.exe')
    options.add_argument('--headless')
    options.add_argument("--start-with-cache")
    options.add_argument("0")
    options.add_argument("--disk-cache-size=0")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Firefox(options=options)
    return driver

def save_data_to_disk(data, filename):
    if os.path.isfile(filename):
        existing_df = pd.read_csv(filename)
        data = pd.concat([existing_df, data], ignore_index=True)
        os.remove(filename)
        data.to_csv(filename,index=False)
    else:
        data.to_csv(filename, mode='w', header=True, index=False)

In [3]:
df= pd.read_csv('location_req.csv')

In [4]:
driver = driver_setup()
driver.implicitly_wait(5)
driver.get('https://www.google.com/maps/')

The version of firefox cannot be detected. Trying with latest driver version


In [5]:
totlis = [3785+420+1684+1407, 7571]
st_rng = totlis[0]
end_rng = totlis[1]
total_rng = abs(end_rng - st_rng)

pbar = tqdm(total=total_rng, desc='Searching', unit='query')

for sub in range(st_rng, end_rng):

    pbar.update(1)
    
    driver.find_element(By.XPATH, '//*[@id="searchboxinput"]').send_keys(df['unique_locstions'][sub])
    driver.find_element(By.XPATH, '//*[@id="searchbox-searchbutton"]').click()
    time.sleep(3)
    current_url = driver.current_url

    pattern = r'@(-?\d+\.\d+),(-?\d+\.\d+)'
    match = re.search(pattern, current_url)

    if match:
        lat = float(match.group(1))
        lon = float(match.group(2))
        
        main ={
            'Search_string': df['unique_locstions'][sub],
            'lat' : lat,
            'lon':lon
        }

        tempdf = pd.DataFrame(main, index=[0])
        tempfilename = 'Lat_and_lon_of_locations_2.csv'
        save_data_to_disk(tempdf, tempfilename)
        tempdf = pd.DataFrame()
    try:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.lSDxNd button'))).click()
    except:
        driver.quit()
        driver = driver_setup()
        driver.implicitly_wait(5)
        driver.get('https://www.google.com/maps/')

Searching:   0%|          | 0/275 [00:00<?, ?query/s]

Searching: 100%|██████████| 275/275 [17:10<00:00,  3.70s/query]